In [1]:
import pandas as pd
import syft as sy
import numpy as np
from syft.core.adp.entity import DataSubject

### Loading the dataset

In [2]:
data = pd.read_csv("../datasets/it - feb 2021.csv")[0:100]
data.head()

/Users/areopagus/opt/anaconda3/envs/syft/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,6,1,1,Imports,381,Italy,...,Australia,NaN,940510,Chandeliers and other electric ceiling or wall...,0,NaN,NaN,6.0,1899,0
1,HS,2021,202102,February 2021,6,1,1,Imports,381,Italy,...,Indonesia,NaN,292700,"Diazo-, azo- or azoxy compounds",0,NaN,NaN,33600.0,134879,0
2,HS,2021,202102,February 2021,6,1,1,Imports,381,Italy,...,Latvia,NaN,292700,"Diazo-, azo- or azoxy compounds",0,NaN,NaN,33127.0,314410,0
3,HS,2021,202102,February 2021,6,1,1,Imports,381,Italy,...,Latvia,NaN,292990,Nitrogen-function compounds; n.e.s. in chapter...,0,NaN,NaN,4.0,4142,0
4,HS,2021,202102,February 2021,6,1,1,Imports,381,Italy,...,Malaysia,NaN,293040,Organo-sulphur compounds; methionine,0,NaN,NaN,51000.0,104889,0


### Logging into the domain

In [3]:
it = sy.login(email="info@openmined.org",
             password="changethis",
             port=8082)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to http://localhost:8082... done! 	 Logging into competent_hassabis... done!


### Upload the dataset to Domain node

In [4]:
# We will upload only the first few rows
# All these three columns are of `int` type

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT. We need to create flags for this or something
data_batch = ((np.array(list(data['Trade Value (US$)'])) / 100000)[0:10]).astype(np.int32)
trade_partners = ((list(data['Partner'])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(data_batch)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(0, 3, entities="bob").tag("trade_flow")

In [5]:
it.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description="""A collection of reports from iStat's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [6]:
it.datasets

Idx,Name,Description,Assets,Id
[0],Italy Trade Data - First few rows,A collection of reports from iStat's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Italy Trade""] -> Tensor",40547bad-d25b-43c4-b3c0-62293e273282


### Create a Data Scientist User

In [7]:
it.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget":200
    }
)

### Accept/Deny Requests to the Domain

In [18]:
it.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[],,<UID: 9cac4c7d09b341948d4886ae694da367>,<UID: 0c3bac4e8cc8e13ac093a3146d57e517>,
1,"[#Italy Trade, __add__]",,<UID: 4a3a70fba80343fabba363e93d5b72b3>,<UID: b979466d04d14c0ebf2c03e8dc7c2f48>,
2,[],,<UID: 8d1324f5dd6549fa9e8613165e8802ed>,<UID: ee99fd7bf376bab9608b29241f4fe5f1>,
3,[],,<UID: eb021c96784d4f63bbd18eac9285ae08>,<UID: ede062b301fea6fc877f271c18b1ac20>,


In [19]:
it.requests[-1].accept()